In [ ]:
import numpy as np

array_1 = np.random.rand(5)
array_2 = np.random.rand(5)

result = array_1 + array_2


print('array_1: ', array_1)
print('array_2: ', array_2)
print('result: ', result)

array_1:  [0.70436727 0.70169767 0.39008446 0.8444052  0.58743137]
array_2:  [0.3043458  0.08829642 0.56558983 0.59365593 0.68106689]
result:  [1.00871307 0.7899941  0.95567429 1.43806114 1.26849826]


In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

array_1 = np.random.rand(5).astype(np.float32)
array_2 = np.random.rand(5).astype(np.float32)

# Allocate GPU memory for arrays
array_1_gpu = cuda.mem_alloc(array_1.nbytes)
array_2_gpu = cuda.mem_alloc(array_2.nbytes)
result_gpu = cuda.mem_alloc(array_1.nbytes)


#Copy data to GPU memory
cuda.memcpy_htod(array_1_gpu, array_1)
cuda.memcpy_htod(array_2_gpu, array_2)



#Kernel
kernel_code = """
__global__ void add_arrays(float *array1, float *array2, float *result){
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  result[tid] = array1[tid] + array2[tid];
}
"""


mod = SourceModule(kernel_code)
add_arrays = mod.get_function('add_arrays')


block = (5, 1, 1)
grid =(5, 1)


add_arrays(array_1_gpu, array_2_gpu, result_gpu, block=block, grid=grid)

result = np.empty_like(array_1)

cuda.memcpy_dtoh(result, result_gpu)



print('array_1: ', array_1)
print('array_2: ', array_2)
print('result: ', np.round(result, 4))



array_1:  [0.26991558 0.33843374 0.27491134 0.0345143  0.45463845]
array_2:  [0.6686083  0.88990337 0.9188843  0.39154232 0.7492851 ]
result:  [0.9385 1.2283 1.1938 0.4261 1.2039]


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
